In [7]:
import numpy as np
import cupy as cp
import pycuda.driver as drv
import torch


In [2]:
s = cp.cuda.Stream()
cp.cuda.driver.ctxGetCurrent()

94446133123200

In [3]:
import pycuda.autoprimaryctx
import pycuda.gpuarray as garray

In [38]:
import sympy as sp

In [45]:
value = np.random.randn(10,15).tolist()

a = np.asarray(value, dtype=float)
# a_pycuda = garray.to_gpu(value, float)
# a_cp = cp.asarray(value, float)
# a_torch = torch.as_tensor(value, dtype=float, device='cuda')

In [58]:
b = a.view(np.recarray)

In [64]:
dtype= np.dtype({'names':[ 'x', 'y', 'z'], 'formats': [np.float_, np.int_, np.float_]})

In [68]:
a = np.random.randn(10, 3).astype(dtype).view(np.recarray)

In [75]:
b = cp.empty(a.shape, dtype=dtype)
b.set(a)

In [78]:
cp.abs(b)

TypeError: Wrong type ((<class 'numpy.void'>,)) of arguments for cupy_absolute

In [70]:
cp.asarray(a)

ValueError: Unsupported dtype (numpy.record, [('x', '<f8'), ('y', '<i8'), ('z', '<f8')])

In [63]:
cp.asarray(b)

array([[-0.28322866,  1.31046587, -0.28454656,  1.81674908, -0.01119491,
        -0.81663423, -0.05573927,  1.66537851,  0.09989158, -1.05034312,
        -0.53409308,  0.91470307, -1.98390357, -0.06409027,  0.20184655],
       [ 2.29723725, -2.05374366,  1.62900998, -1.49553315, -0.64859651,
        -0.30070889, -0.37873686,  0.73233141, -0.3831772 ,  1.59960095,
        -0.1310352 , -0.02048398,  0.66022252, -1.13966363,  0.96602675],
       [-0.05906964,  0.40385257,  0.60172368, -0.15493863, -1.5256004 ,
         0.342295  , -1.41092024,  0.77355015, -2.78972553, -1.0508179 ,
         0.72802405,  0.35254193, -0.15767821, -0.25352298,  0.16065977],
       [ 0.5969111 ,  0.23037335,  1.62221741, -0.96698191, -1.49819656,
        -1.04113677,  0.10177433,  0.16266169,  0.73465572,  1.65144004,
        -2.07822497,  0.7938752 ,  0.08807907, -1.06902852,  1.60059562],
       [-0.23965714,  0.40095116,  0.79233786, -0.33191428,  0.85934906,
         0.89154937, -0.39371164, -1.02311937, 

In [62]:
%timeit 'x' in b.dtype.fields

TypeError: argument of type 'NoneType' is not iterable

In [61]:
%timeit 'x' in  b.dtype.names

148 ns ± 0.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [57]:
a.view(np.recarray).field('x')

TypeError: 'NoneType' object is not subscriptable

In [19]:
a.__array_interface__

{'data': (94446122710080, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (5,),
 'version': 3}

In [14]:
a_pycuda.__cuda_array_interface__

{'shape': (5,),
 'strides': (8,),
 'data': (139959941990912, False),
 'typestr': '<f8',
 'stream': None,
 'version': 3}

In [15]:
a_cp.__cuda_array_interface__

{'shape': (5,),
 'typestr': '<f8',
 'descr': [('', '<f8')],
 'stream': 1,
 'version': 3,
 'strides': None,
 'data': (139959941989376, False)}

In [13]:
a_torch.__cuda_array_interface__

{'typestr': '<f8',
 'shape': (5,),
 'strides': None,
 'data': (139956536213504, False),
 'version': 2}

In [22]:
cp.asarray(a_pycuda)
cp.asarray(a_torch)
cp.asarray(a)

array([0., 0., 0., 0., 0.])

In [21]:
torch.as_tensor(a)
torch.as_tensor(a_cp)
torch.as_tensor(a_pycuda)

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
torch.as_tensor(a)
torch.as_tensor(a_cp)
torch.as_tensor(a_pycuda)

In [24]:
garray.to_gpu(a)
garray.to_gpu(a_cp)

TypeError: src must be GPUArray or ndarray

In [8]:
pycuda.autoprimaryctx.context

In [13]:
dev = torch.cuda.device(0)

In [29]:
import numpy
a = numpy.random.randn(4,4)
a_gpu = drv.mem_alloc(a.nbytes)
drv.memcpy_htod(a_gpu, a)

In [34]:
garray.GPUArray(a.shape, dtype=a.dtype, gpudata=a_gpu)

array([[-0.27258862,  0.18403886, -0.97767967, -0.13758831],
       [-0.82263791,  0.91515323,  1.31963314,  1.6223802 ],
       [ 0.84822896, -2.19246203, -1.25890748, -1.38398629],
       [ 0.49990518,  0.1539747 , -1.62959918, -0.36374604]])

In [80]:
np.float_

numpy.float64

In [79]:
MyType = np.dtype({'names': ['x', 'y', 'z'], 'formats': [np.float_]*3})
preamble = """
struct MyType {
    double x, y, z; 
}
"""
code = r'''
template<typename T>
__global__ void fx3(T* arr, int N) {
    unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < N) {
        arr[tid]->x = arr[tid]->x * 3;
    }
}
'''
name_exp = ['fx3<MyType>']
mod = cp.RawModule(code=code, options=('-std=c++11',),
    name_expressions=name_exp)
kern = mod.get_function(name_exp[0])  # compilation happens here
N=10
a = cp.random.randn(10,3).view(MyType)
kern((1,), (N,), (a, N))
a

CompileException: __nv_name_map(2): error: identifier "MyType" is undefined

__nv_name_map(2): error: Error in parsing name expression for lowered name lookup. Input name expression was: " fx3 < MyType >"

2 errors detected in the compilation of "/tmp/tmpxpf1s0fw/14aa8e4c83082b0664b4477201162891ea1b930a.cubin.cu".


In [88]:
MyType = np.dtype({'names': ['x', 'y', 'z'], 'formats': [np.float_]*3})
preamble = """
struct mytype {float x,y,z,w,v;};
"""
mod = cp.ElementwiseKernel(
    "mytype a",
    "mytype b",
    "a->x = b->x * 2",
    "double",
    preamble=preamble
)

TypeError: data type 'mytype' not understood

In [ ]:
kern = mod.get_function(name_exp[0])  # compilation happens here
N=10
a = cp.random.randn(10,3).view(MyType)
kern((1,), (N,), (a, N))
a

In [1]:
import numpy as np
import cupy as cp

code = r"""
    struct mytype {float x,y,z,w,v;};
    template <typename T>
    __global__ void affect(T* const __restrict__ output,
                         const T arg0, const T arg1,
                         const size_t nelements) {
        ptrdiff_t i = blockIdx.x * blockDim.x + threadIdx.x;
        if (i >= nelements)
            return;
        output[i] = (i%2 == 0 ? arg0 : arg1);
    }
"""
module = cp.RawModule(code=code,
                      name_expressions=('affect<mytype>',),
                      options=('-std=c++11',))
kernel = module.get_function('affect<mytype>')

mytype = np.dtype( { 'names': ['x', 'y', 'z', 'w', 'v'],
                     'formats': [np.float32]*5 } )

arg0 = np.random.rand(5, 5).astype(np.float32).view(mytype)
arg1 = np.random.rand(5, 5).astype(np.float32).view(mytype)

N = 512
h_output = np.empty(dtype=mytype, shape=N)
d_output = cp.asarray(h_output.view(np.float32))

grid = (N,)
block = (256,)
args = (d_output, arg0, arg1, np.uint64(N))
kernel(grid, block, args)

h_output[0::2] = arg0
h_output[1::2] = arg1

np.testing.assert_array_equal(h_output,
                              cp.asnumpy(d_output).view(float5))

TypeError: You are trying to pass a numpy.ndarray of shape (5, 1) as a kernel parameter. Only numpy.ndarrays of size one can be passed by value. If you meant to pass a pointer to __global__ memory, you need to pass a cupy.ndarray instead.

In [87]:
arg0

array([(0.8242026, 0.9768041, 0.40528548, 0.9857719, 0.7546456)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('w', '<f4'), ('v', '<f4')])